# Dynamic Factor Analysis for Stress Period Detection

Author: S S K Pavan, Nitin Gautam

Roll No. 2020HES7045

In [11]:

import types
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from statsmodels.tools.eval_measures import rmse


import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("MultiStat_project.csv")

In [3]:
data["year"] = pd.to_datetime(data.year, format = "%Y")
indexed_data = data.set_index(["country", "year"], inplace = False)

In [4]:
# list of countries in the dataset
country_lst = data.country.drop_duplicates()
len(country_lst)
# we do not have a balanced panel here

43

## Modelling Stratergy

For each country we estimate a seperate dynamic factor model. We follow the steps:

Step 1:

estimating fcators for each country

Step 2:

taking these factors as panel data, running a logistic regression

For each country we esitmate the following set of equations
$$
Y_{t} = \Lambda f_{t} + \epsilon_t \\
f_{t} = \beta f_{t-1} + u_t
$$

1. $Y_t$ contains all the explanatory variables in the dataset
2. $\Lambda$ is the matrix of factor loadings
3. $f_t$ is the matrix of the common factors which has a VAR(1) specification 
4. $u_t$ is the error term which is modelled as iid Multivariate Normal RV 


for each country we estimate $f_{it}$. We stack them for each country

### Predicting country Stress using the estimated factors in a logit Model  

Now we estimate our logit model which predicts stress:

$$
stress_{it} = \gamma f_{it} + stress_{it-1} + \alpha_{i} + \phi_{it} 
$$

Where $\alpha_{i}$ is the country specific fixed effect

## Model Specification

we need to specify all the details of the specific dynamic factor model that we want to estimate. We need to decide on
1. how many factors to include
2. which variables load on which factors 
3. the order of the vector autoregression that the factor evolves according to.

**Factor specification**

we will just choose an ad-hoc structure for the number of factors to estimate:

- Two country factors (i.e. factors that load on all variables) that jointly evolve according to a VAR(1)
- One group-specific factor (i.e. factors that load only on variables in their group) evlove according to an AR(1)

In the `DynamicFactorMQ` model, the basic factor structure is specified using the `factors` argument, which must be one of the following:

- An integer, which can be used if one wants to specify only that number of global factors. This only allows a very simple factor specification.
- A dictionary with keys equal to observed variable names and values equal to a list of factor names. Note that if this approach is used, the all observed variables must be included in this dictionary.

Since we have a complex factor specification we will use the second method and specify our fators in the form of a dictionary.

```python
factors = {
    'Real Personal Income': ['Global', 'Output and Income'],
    ...
}
```

In [5]:
data.columns

Index(['Unnamed: 0', 'country', 'year', 'cpi', 'dyn_gdp', 'dyn_gdp_china',
       'dyn_GDP_US', 'interest_rate_US', 'oil_yoy', 'dyn_consum',
       'diff_priv_credit_gdp', 'net_lending', 'public_debt',
       'interest_on_debt', 'overvaluation', 'ca_balance', 'dyn_fix_cap_form',
       'dyn_export_share', 'diff_unempl', 'dyn_prod_dol', 'VIX', 'GDP_per_cap',
       'developed', 'crisis_next_year', 'crisis_next_period',
       'crisis_first_year'],
      dtype='object')

In [6]:
factors = {'interest_rate_US': ["country", "Macro"],
            'dyn_GDP_US':      ["country", "Macro"],
            'dyn_gdp_china':   ["country", "Macro"],
            'oil_yoy':         ["country", "Macro"],
            'VIX'    :         ["country", "Macro"],
            'dyn_gdp' :        ["country", "Macro"],
            'GDP_per_cap' :    ["country", "Macro"],
            'overvaluation' :  ["country", "domestic"],
            'ca_balance'    :  ["country", "domestic"],
            'dyn_export_share':["country", "domestic"],
            'dyn_fix_cap_form':["country", "domestic"],
            'cpi'             :["country", "domestic"],
            'dyn_consum'      :["country", "domestic"],
            'diff_priv_credit_gdp':["country", "financial"],
            'public_debt'     :["country", "fiscal"],
            'net_lending'     :["country", "fiscal"],
            'interest_on_debt':["country", "fiscal"],
            'diff_unempl'     :["country", "labor"],
            'dyn_prod_dol'    :["country", "labor"]
            
            }

Unemployment rate, change in p.p.

**Factor multiplicities**

 `factor_multiplicities` argument allows setting up of multiple factors that evolve together jointly.

The `factor_multiplicities` argument we specify a dictionary with keys equal to factor names (from the `factors` argument) and values equal to an integer.

Here, we want all of the group factors to be univariate, while we want a bivariate set of global factors. Therefore, we only need to specify the `{'Global': 2}` part, while the rest will be assumed to have multiplicity 1 by default.

In [7]:
factor_multiplicities = {'country': 2}

**Factor orders**

We need to specify the lag order of the (vector) autoregressions that govern the dynamics of the factors.

The `factor_orders` argument defaults to `1`, but if it is specified then it must be one of the following:

- An integer, which can be used if one wants to specify the same order of (vector) autoregression for each factor.
- A dictionary with keys equal to factor names (from the `factors` argument) or tuples of factor names and values equal to an integer. Include in this dictionary only the factors that have order greater than 1.

Since we are specifying a parsimonious model of VAR(1), we let the value be its default


**Creating the model**
Now that we have a simple factor specification, let us discuss the primary arguments in the `DynamicFactorMQ` model class

1. `endog` and `endog_quarterly`

   These arguments are used to pass the observed variables to the model. There are two ways to provide the data:
   
   1. We will pass all of your observed montly variables to the `endog` variable and not include the `endog_quarterly` argument. Our aim is dimesnonality reduction. rather if this was a prediction exercise we would pass the less freq variable that we are predicting into `endog_quarterly`

2. `factors`, `factor_orders`, and `factor_multiplicities`

3. `idiosyncratic_ar1`

   The `DynamicFactorMQ` model allows the idiosyncratic disturbance terms to be modeled as independent AR(1) processes or as iid variables. The default is `idiosyncratic_ar1=True`, which models a AR(1) process for the esrror terms. We set it to `false`. Setting it to True can help in modeling some of the idiosyncratic serial correlation for forecasting.


In [8]:
# empty list to collect all the estimated factors
stress_variables = [] 

for country in country_lst:
    # run the model seperately for each country
    model = sm.tsa.DynamicFactorMQ(
                                    indexed_data.loc[country, [i for i in factors.keys()]],
                                    factors=factors, 
                                    factor_orders = 1,
                                    factor_multiplicities = factor_multiplicities
                                    )
    # get the results of the estimation
    results = model.fit(maxiter = 5000)
    # get the factors
    fact = results.factors.smoothed
    # rename the index for easy concatination
    fact.index.name = "year"
    # add multiindex
    indexed_factors = pd.concat({country: fact}, names=['country'])
    stress_variables.append(indexed_factors)




C:\Users\pavan\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Users\pavan\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Users\pavan\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Users\pavan\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Users\pavan\anaconda3\lib\site-packages\statsmodels\tsa\statespace\dynamic_factor_mq.py:2680: Use

In [14]:
# concat all the factors for individual countries in the list stress_variables 
all_factors = pd.concat(stress_variables, axis = 0)

# concat with the crisis_next_year
pred_df = pd.concat([indexed_data.loc[:,["crisis_next_year"]], all_factors], axis= 1)

In [17]:
# save file to exccel sheet
pred_df.to_excel("data_with_factors.xlsx")

## Panel Data Logit

In [24]:
collapsed = pred_df.reset_index()
collapsed.head()

,country,year,crisis_next_year,country.1,country.2,Macro,domestic,fiscal,labor,financial
0,Albania,1997-01-01,1,-3.880345,-0.288062,-1.120430,-0.395629,9.455013,-4.550029,-0.156959
1,Albania,1998-01-01,0,-3.912228,-0.840525,0.050421,-0.566456,5.909447,0.197792,-0.599430
2,Albania,1999-01-01,1,-3.167257,-3.903692,-0.001344,0.867782,3.468259,0.599404,-1.288816
3,Albania,2000-01-01,0,-2.067584,-2.267763,-0.001183,0.889022,2.038466,-1.568611,-0.730585
4,Albania,2001-01-01,0,-1.344817,-1.779261,0.000037,0.321831,0.603040,0.084477,-0.468229


In [26]:
collapsed = collapsed.rename(columns = {"country.1": "global_1",
                            "country.2": "global_2"})

In [64]:
# run logit regerssion with clustered standard errors
formula = "crisis_next_year ~ crisis_next_year.shift(1) + global_1 + global_2 + Macro + domestic + fiscal + labor + financial + C(country)"

# covariance options abandoned for now
cov_options = {'maxlags': 2}
logit_model = smf.logit(formula, data = collapsed ).fit(method = "bfgs", maxiter=500, cov_type = "HAC", cov_kwds = cov_options)

Optimization terminated successfully.
         Current function value: 0.199894
         Iterations: 213
         Function evaluations: 214
         Gradient evaluations: 214


In [67]:
info_dict={'Log-likelihood Ratio' : lambda x: f"{x.llf:.3f}",
           'Log-likelihood Null' : lambda x: f"{x.llnull:.3f}",
           'Pseudo-R sq' : lambda x: f"{x.prsquared:.3f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}",
            }

results_table = summary_col(results= [logit_model],
                            float_format='%0.3f',
                            stars = True,
                            model_names=["stress factors "],
                            regressor_order= ["crisis_next_year.shift(1)" , "global_1", "global_2", "Macro", "domestic", "fiscal", "labor"],
                            info_dict=info_dict,
                            drop_omitted=  True
                            )

results_table.add_title("Logit model using Dynamic fcators")

print(results_table.as_latex())
# print(results_table)

\begin{table}
\caption{Logit model using Dynamic fcators}
\label{}
\begin{center}
\begin{tabular}{ll}
\hline
                            & stress factors   \\
\hline
crisis\_next\_year.shift(1) & 2.910***         \\
                            & (0.277)          \\
global\_1                   & 0.286***         \\
                            & (0.063)          \\
global\_2                   & 0.008            \\
                            & (0.071)          \\
Macro                       & -0.011           \\
                            & (0.092)          \\
domestic                    & -0.162           \\
                            & (0.122)          \\
fiscal                      & 0.038            \\
                            & (0.107)          \\
labor                       & 0.239**          \\
                            & (0.118)          \\
Log-likelihood Ratio        & -197.096         \\
Log-likelihood Null         & -443.716         \\
Pseudo-R sq                 & 0.55